In [ ]:
import os
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, ToolMessage, AIMessage, SystemMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, END
from langchain_core.tools import tool
from typing import TypedDict, Annotated
import operator

def read_md_file(filename: str) -> str:
    """Read a markdown file from the 'config' folder and return its content as a string."""
    config_path = os.path.join("config", filename)
    with open(config_path, "r", encoding="utf-8") as f:
        return f.read()
    
def format_message(message_object) -> str:
    """
    Converts a LangChain message object into a human-readable string.

    Args:
        message_object: An instance of AIMessage, HumanMessage, or ToolMessage.

    Returns:
        A formatted string with a prefix (System:, User:, Tool:).
    """
    if isinstance(message_object, AIMessage):
        return f"System: {message_object.content}"
    elif isinstance(message_object, HumanMessage) or isinstance(message_object, ToolMessage):
        return f"User: {message_object.content}"
    else:
        # Handle any other message types gracefully
        return f"Unknown: {str(message_object.content)}"
    
# Define the state of our graph
class State(TypedDict):
    system_prompt: str
    formatted_messages: list[str]
    messages: Annotated[list, operator.add]

# --- INTERNAL INTERRUPT LOGIC ---

# This tool runs when called by the model.
@tool()
def human_tool(query: str):
    """You will be responsible for answering any clarification questions that the LLM may have. eg. What is the weight of the package? Where are you shipping from and to? etc"""
    print(f"\n🤖 AI is requesting Human assistance for: '{query}'")
    human_input = input("🧑‍💻 Your response: ")
    print (f"🧑‍💻 Human provided: '{human_input}'\n")
    return human_input

def stateful_human_tool_wrapper(state: State) -> dict:
    """
    This "wrapper" function handles all the LangGraph state logic.
    """
    print("=== Human Tool Wrapper Invoked ===")
    # 1. Extract the query from the state
    last_ai_message = state['messages'][-1]
    tool_call = last_ai_message.tool_calls[0]
    query = tool_call['args']['query']
    
    # Save ethe System feedback in the state
    state["formatted_messages"].append(f"System: {query}")
    
    # 2. Call your original, simple function with just the query
    human_response = human_tool.invoke(query)
    
    # Save ethe user feedback in the state
    state["formatted_messages"].append(
        format_message(ToolMessage(content=human_response, tool_call_id=tool_call['id']))
    )
    
    # 3. Return the state update in the correct format
    return {
        "messages": [
            ToolMessage(content=human_response, tool_call_id=tool_call['id'])
        ]
    }

# Initialize the Ollama model and bind the tool
# llm = ChatOpenAI(model="gpt-4o")
llm = ChatOllama(model="Mistral", temperature=0)
tools = [human_tool]
model = llm.bind_tools(tools)

# --- GRAPH NODES ---
def chatbot(state: State):
    """The chatbot node that calls the LLM."""
    print(100*"=")
    
    # This print statement is now safe and handles any message type with a .content attribute.
    # print(f"---🤖 Chatbot thinking... Reacting to message of type: {type(last_message).__name__}---")
    print(f"---🤖 Chatbot thinking... Reacting to message of type: {state["formatted_messages"]}--")

    # The graph state `messages` already contains the full history.
    # We just need to add the system prompt for this specific invocation.
    # messages_for_llm = [SystemMessage(content=state["system_prompt"])] + state["messages"]
    messages_for_llm = [SystemMessage(content=state["system_prompt"])] + state["messages"]
    print(100*"-")
    
    response = model.invoke(messages_for_llm)  
    
    print(f"LLM Response: {response}")
    print(100*"=")
        
    return {"messages": [response]}

# The tool node for the human input
human_node = ToolNode(tools)

# --- GRAPH DEFINITION ---
graph = StateGraph(State)

graph.add_node("chatbot", chatbot)
graph.add_node("human", stateful_human_tool_wrapper)

def router(state: State):
    """Router to decide the next step."""
    last_message = state["messages"][-1]
    if isinstance(last_message, AIMessage):
        print("The last message was from the AI.")
        # Check if the AI message has tool calls
        if last_message.tool_calls:
            print("AI is calling a tool. Routing to the 'human' node.")
            return "human"
        else:
            print("AI responded without tools. Ending the turn.")
            return END

    elif isinstance(last_message, ToolMessage):
        print("The last message was a Tool's output. Routing back to the 'chatbot'.")
        return "chatbot"

    elif isinstance(last_message, HumanMessage):
        print("The last message was from the User. Routing to the 'chatbot'.")
        return "chatbot"

    else:
        # Fallback or end the process if the message type is unexpected
        print("Unknown message type. Ending the turn.")
        return END

# Set the entry point and edges
graph.set_entry_point("chatbot")
graph.add_conditional_edges("chatbot", router)

# **THE KEY CHANGE IS HERE**
# After the human provides input, loop back to the chatbot
# so it can process the new information.
graph.add_edge("human", "chatbot")

# Compile the graph
memory = MemorySaver()
app = graph.compile(checkpointer=memory)


In [2]:
# --- UI DRIVER ---
# This loop is still necessary to start each new conversation turn.
print("Chatbot is ready. Type 'quit' to exit.")

#I want to ship a computer to new york

config = {"configurable": {"thread_id": "thread_123"}}

while True:
    user_input = input("👤 You: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break

    # This kicks off the graph execution for the current turn
    for chunk in app.stream(
        {"messages": [user_input],
         "formatted_messages": [f"User: {user_input}"],
         "system_prompt": read_md_file("shipping_rate_lookup.md")},
        config=config,
    ):
        # Print the final output from the chatbot after the whole loop is done
        if END in chunk:
            final_message = chunk[END]['messages'][-1]
            if final_message.content:
                print(f"🤖 AI: {final_message.content}")

Chatbot is ready. Type 'quit' to exit.
---🤖 Chatbot thinking... Reacting to message of type: ['User: ship a computer to New Jersey']--
----------------------------------------------------------------------------------------------------
LLM Response: content=' [{"name":"human_tool","arguments":{"query":"What is the weight of the computer in pounds?"}}]' additional_kwargs={} response_metadata={'model': 'Mistral', 'created_at': '2025-10-10T06:27:38.812303Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4633703000, 'load_duration': 29060417, 'prompt_eval_count': 491, 'prompt_eval_duration': 3723411000, 'eval_count': 26, 'eval_duration': 877202833, 'model_name': 'Mistral'} id='run--ee5aa160-7b71-4845-8393-0302169b1a4e-0' usage_metadata={'input_tokens': 491, 'output_tokens': 26, 'total_tokens': 517}
The last message was from the AI.
AI responded without tools. Ending the turn.
Goodbye!
